# Implement PKCS\#7 padding

In [12]:
def pkcs7(data: bytes, *, block_size: int = 16) -> bytes:
    padding = block_size - (len(data) % block_size)
    return data + bytes([padding for x in range(padding)])

In [17]:
pkcs7(b"YELLOW SUBMARINE", block_size=20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# Implement CBC mode

In [19]:
%%capture
!pip install pycryptodome
from Crypto.Cipher import AES
import base64
import binascii
import numpy as np

In [20]:
def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()

In [ ]:
key = pkcs7(b"YELLOW SUBMARINE", block_size=16)
iv = b"\x00" * 16

with open("set2/10.txt") as fd:
    data = fd.read()
data = data.replace("\n", "")